In [1]:
!pip install datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you 

In [2]:
import torch
import torch.nn as nn
from transformers import SiglipVisionModel, SiglipProcessor, AutoModel, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from PIL import Image

2025-04-18 09:14:52.553841: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744967692.814627      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744967692.883052      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
siglip_vision = SiglipVisionModel.from_pretrained("google/siglip-base-patch16-224")
deberta_text = AutoModel.from_pretrained("microsoft/deberta-v3-base")
siglip_processor = SiglipProcessor.from_pretrained("google/siglip-base-patch16-224")
deberta_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
ds = load_dataset("nlphuji/flickr30k")

README.md:   0%|          | 0.00/641 [00:00<?, ?B/s]

flickr30k.py:   0%|          | 0.00/2.51k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/506M [00:00<?, ?B/s]

0001.parquet:   0%|          | 0.00/502M [00:00<?, ?B/s]

0002.parquet:   0%|          | 0.00/506M [00:00<?, ?B/s]

...

0005.parquet:   0%|          | 0.00/495M [00:00<?, ?B/s]

0006.parquet:   0%|          | 0.00/495M [00:00<?, ?B/s]

0007.parquet:   0%|          | 0.00/497M [00:00<?, ?B/s]

0008.parquet:   0%|          | 0.00/289M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/31014 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    test: Dataset({
        features: ['image', 'caption', 'sentids', 'split', 'img_id', 'filename'],
        num_rows: 31014
    })
})

In [6]:
import random
random.seed(42)
total_examples = len(ds['test'])
indices = list(range(total_examples))
random.shuffle(indices)

In [7]:
train_size = 15000
test_size = 4000
search_size = 11000
train_indices = indices[:train_size]
test_indices = indices[train_size:train_size + test_size]
search_indices = indices[train_size + test_size:train_size + test_size + search_size]

train_ds = ds['test'].select(train_indices)
test_ds = ds['test'].select(test_indices)
search_ds = ds['test'].select(search_indices)

In [8]:
class MLPProjection(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=512, output_dim=512):
        super(MLPProjection, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(0.1)
        self.layer2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.layer1(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = nn.functional.normalize(x, p=2, dim=-1)
        return x

In [9]:
    class VLM(nn.Module):
        def __init__(self, vision_encoder, text_encoder):
            super(VLM, self).__init__()
            self.vision_encoder = vision_encoder
            self.text_encoder = text_encoder
            self.image_projection = MLPProjection(input_dim=768, hidden_dim=512, output_dim=512)
            self.text_projection = MLPProjection(input_dim=768, hidden_dim=512, output_dim=512)
    
        def forward(self, pixel_values, input_ids, attention_mask):
            vision_outputs = self.vision_encoder(pixel_values)
            image_features = vision_outputs.pooler_output
            text_outputs = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
            text_features = text_outputs.last_hidden_state[:, 0, :]
            image_proj = self.image_projection(image_features)
            text_proj = self.text_projection(text_features)
            return image_proj, text_proj
    
    
    model = VLM(siglip_vision, deberta_text)

In [ ]:
def siglip_loss(image_features, text_features, temperature=0.1):
    image_features = nn.functional.normalize(image_features, p=2, dim=-1)
    text_features = nn.functional.normalize(text_features, p=2, dim=-1)

    logits = torch.matmul(image_features, text_features.T) / temperature
    batch_size = logits.shape[0]

    labels = torch.eye(batch_size, device=logits.device)

    loss = nn.functional.binary_cross_entropy_with_logits(logits, labels)
    return loss

In [ ]:
def collate_fn(batch):
    images = [item['image'] for item in batch]

    captions = []
    for item in batch:
        if isinstance(item['caption'], str):
            captions.append(item['caption'])
        elif isinstance(item['caption'], list):
            captions.append(item['caption'][0])  

    image_inputs = siglip_processor(images=images, return_tensors="pt", padding=True)

    text_inputs = deberta_tokenizer(captions, padding=True, truncation=True, return_tensors="pt")

    return image_inputs['pixel_values'], text_inputs['input_ids'], text_inputs['attention_mask']


train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)
search_loader = DataLoader(search_ds, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [12]:
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from torch.cuda.amp import GradScaler, autocast
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=3e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=10)
scaler = GradScaler()

for epoch in range(100):
    model.train()
    total_loss = 0
    start_time = time.time()
    for pixel_values, input_ids, attention_mask in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        pixel_values = pixel_values.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        with autocast():
            image_proj, text_proj = model(pixel_values, input_ids, attention_mask)
            loss = siglip_loss(image_proj, text_proj, temperature=0.1)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()

    scheduler.step()
    epoch_time = time.time() - start_time
    print(f"Epoch {epoch+1}, Training Loss: {total_loss / len(train_loader):.4f}")
    print(f"Epoch {epoch+1}, Learning Rate: {scheduler.get_last_lr()[0]:.6f}")
    print(f"Epoch {epoch+1}, Time: {epoch_time:.2f} seconds")

    model.eval()
    test_loss = 0
    with torch.no_grad():
        for pixel_values, input_ids, attention_mask in test_loader:
            pixel_values = pixel_values.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            with autocast():
                image_proj, text_proj = model(pixel_values, input_ids, attention_mask)
                loss = siglip_loss(image_proj, text_proj, temperature=0.1)
            test_loss += loss.item()
    print(f"Epoch {epoch+1}, Test Loss: {test_loss / len(test_loader):.4f}")

    checkpoint_path = 'model_checkpoint.pth'
    torch.save(model, checkpoint_path)
    print(f"Model saved for epoch {epoch+1} at {checkpoint_path}")


/tmp/ipykernel_19/2223608671.py:5: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1:   0%|          | 0/469 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipykernel_19/2223608671.py:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1: 100%|██████████| 469/469 [05:31<00:00,  1.42it/s]


Epoch 1, Training Loss: 0.1267
Epoch 1, Learning Rate: 0.000029
Epoch 1, Time: 331.44 seconds


/tmp/ipykernel_19/2223608671.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1, Test Loss: 0.0985
Model saved for epoch 1 at model_checkpoint.pth


Epoch 2: 100%|██████████| 469/469 [05:21<00:00,  1.46it/s]


...

Epoch 98, Training Loss: 0.0340
Epoch 98, Learning Rate: 0.000027
Epoch 98, Time: 342.35 seconds
Epoch 98, Test Loss: 0.0699
Model saved for epoch 98 at model_checkpoint.pth


Epoch 99: 100%|██████████| 469/469 [05:42<00:00,  1.37it/s]


Epoch 99, Training Loss: 0.0344
Epoch 99, Learning Rate: 0.000029
Epoch 99, Time: 342.56 seconds
Epoch 99, Test Loss: 0.0693
Model saved for epoch 99 at model_checkpoint.pth


Epoch 100: 100%|██████████| 469/469 [05:42<00:00,  1.37it/s]


Epoch 100, Training Loss: 0.0341
Epoch 100, Learning Rate: 0.000030
Epoch 100, Time: 342.52 seconds
Epoch 100, Test Loss: 0.0696
Model saved for epoch 100 at model_checkpoint.pth


In [14]:
def find_caption(image, model):
    model.eval()
    with torch.no_grad():
        image_inputs = siglip_processor(images=image, return_tensors="pt")
        pixel_values = image_inputs['pixel_values'].to(device)
        image_features = model.vision_encoder(pixel_values).pooler_output
        image_proj = model.image_projection(image_features)
        image_proj = nn.functional.normalize(image_proj, p=2, dim=-1)
        return image_proj

def find_image(caption, model):
    model.eval()
    with torch.no_grad():
        text_inputs = deberta_tokenizer(caption, padding=True, truncation=True, return_tensors="pt")
        input_ids = text_inputs['input_ids'].to(device)
        attention_mask = text_inputs['attention_mask'].to(device)
        text_features = model.text_encoder(input_ids, attention_mask).last_hidden_state[:, 0, :]
        text_proj = model.text_projection(text_features)
        text_proj = nn.functional.normalize(text_proj, p=2, dim=-1)
        return text_proj

In [15]:
# Ví dụ test: https://cdn.shopify.com/s/files/1/0624/1746/9697/files/siberian-husky-100800827-2000-9449ca147e0e4b819bce5189c2411188_600x600.jpg?v=1690185264
import requests
from io import BytesIO
def load_image_from_url(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")

In [16]:
_image = load_image_from_url("https://cdn.shopify.com/s/files/1/0624/1746/9697/files/siberian-husky-100800827-2000-9449ca147e0e4b819bce5189c2411188_600x600.jpg?v=1690185264")
caption = "a dog"
image_vector = find_caption(_image, model)
caption_vector = find_image(caption, model)
similarity = torch.cosine_similarity(image_vector, caption_vector)
print(f"Cosine similarity giữa ảnh và caption: {similarity.item()}")

Cosine similarity giữa ảnh và caption: 0.10177205502986908


In [17]:
_image = load_image_from_url("https://images.unsplash.com/photo-1531040630173-7cfb894c8eaa?fm=jpg&q=60&w=3000&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8NHx8d2hpdGUlMjBjYXR8ZW58MHx8MHx8fDA%3D")
caption = "bear is standing"
image_vector = find_caption(_image, model)
caption_vector = find_image(caption, model)
similarity = torch.cosine_similarity(image_vector, caption_vector)
print(f"Cosine similarity giữa ảnh và caption: {similarity.item()}")

Cosine similarity giữa ảnh và caption: -0.062211379408836365
